<img src="https://avatars.githubusercontent.com/u/74911464?s=200&v=4"
     alt="OpenEO Platform logo"
     style="float: left; margin-right: 10px;" />
# openEO Platform UC8
## Create the target variable for the random Forest regression

This notebook creates the target variable for the Random Forest regression.

This approach utilizes the Very-High Resolution (VHR) data in the private PLANETSCOPE collection.

The sampling of 150 study sites with an extent of 16ha each, has been conducted locally before acquiring the VHR data.

### Requirements

In [57]:
import openeo
from openeo.processes import median, sd
import numpy as np
import geopandas as gpd
import xarray as xr
import matplotlib.pyplot as plt
from shapely.geometry.polygon import Polygon
import json

## Connection

Connect to the OpenEO back-end using the OpenEO client

In [2]:
connection = openeo.connect("openeo.cloud")

Authenticate via EGI Check-in (OpenID Connect)

In [3]:
connection.authenticate_oidc("egi")

Authenticated using refresh token.


<Connection to 'https://openeocloud.vito.be/openeo/1.0.0/' with BearerAuth>

The VHR data from `PLANETSCOPE` is stored in the back-end

In [4]:
connection.describe_collection("PLANETSCOPE")

{'cube:dimensions': {'bands': {'type': 'bands',
   'values': ['B1',
    'B2',
    'B3',
    'B4',
    'UDM',
    'UDM2_Clear',
    'UDM2_Snow',
    'UDM2_Shadow',
    'UDM2_LightHaze',
    'UDM2_HeavyHaze',
    'UDM2_Cloud',
    'UDM2_Confidence',
    'dataMask']},
  't': {'extent': ['2016-01-01T00:00:00Z', None], 'type': 'temporal'},
  'x': {'axis': 'x',
   'extent': [-180, 180],
   'reference_system': {'$schema': 'https://proj.org/schemas/v0.2/projjson.schema.json',
    'area': 'World',
    'base_crs': {'coordinate_system': {'axis': [{'abbreviation': 'Lat',
        'direction': 'north',
        'name': 'Geodetic latitude',
        'unit': 'degree'},
       {'abbreviation': 'Lon',
        'direction': 'east',
        'name': 'Geodetic longitude',
        'unit': 'degree'}],
      'subtype': 'ellipsoidal'},
     'datum': {'ellipsoid': {'inverse_flattening': 298.257223563,
       'name': 'WGS 84',
       'semi_major_axis': 6378137},
      'name': 'World Geodetic System 1984',
      'type': 'GeodeticReferenceFrame'},
     'name': 'WGS 84'},
    'bbox': {'east_longitude': 180,
     'north_latitude': 90,
     'south_latitude': -90,
     'west_longitude': -180},
    'coordinate_system': {'axis': [{'abbreviation': 'E',
       'direction': 'east',
       'name': 'Easting',
       'unit': 'metre'},
      {'abbreviation': 'N',
       'direction': 'north',
       'name': 'Northing',
       'unit': 'metre'}],
     'subtype': 'Cartesian'},
    'id': {'authority': 'OGC', 'code': 'Auto42001', 'version': '1.3'},
    'name': 'AUTO 42001 (Universal Transverse Mercator)',
    'type': 'ProjectedCRS'},
   'type': 'spatial'},
  'y': {'axis': 'y',
   'extent': [-90, 90],
   'reference_system': {'$schema': 'https://proj.org/schemas/v0.2/projjson.schema.json',
    'area': 'World',
    'base_crs': {'coordinate_system': {'axis': [{'abbreviation': 'Lat',
        'direction': 'north',
        'name': 'Geodetic latitude',
        'unit': 'degree'},
       {'abbreviation': 'Lon',
        'direction': 'east',
        'name': 'Geodetic longitude',
        'unit': 'degree'}],
      'subtype': 'ellipsoidal'},
     'datum': {'ellipsoid': {'inverse_flattening': 298.257223563,
       'name': 'WGS 84',
       'semi_major_axis': 6378137},
      'name': 'World Geodetic System 1984',
      'type': 'GeodeticReferenceFrame'},
     'name': 'WGS 84'},
    'bbox': {'east_longitude': 180,
     'north_latitude': 90,
     'south_latitude': -90,
     'west_longitude': -180},
    'coordinate_system': {'axis': [{'abbreviation': 'E',
       'direction': 'east',
       'name': 'Easting',
       'unit': 'metre'},
      {'abbreviation': 'N',
       'direction': 'north',
       'name': 'Northing',
       'unit': 'metre'}],
     'subtype': 'Cartesian'},
    'id': {'authority': 'OGC', 'code': 'Auto42001', 'version': '1.3'},
    'name': 'AUTO 42001 (Universal Transverse Mercator)',
    'type': 'ProjectedCRS'},
   'type': 'spatial'}},
 'datasource_type': 'byoc-ID',
 'description': "PlanetScope is one of the satellite constellation operated by Planet. PlanetScope satellite constellation consists of more than 130 small satellites called Doves. Each Dove satellite is a CubeSat made of three cubic units and thus measures only 10 cm x 10 cm x 30 cm. The satellites are launched in groups, which constantly improves mission's characteristics such as revisit times, spatial and spectral resolutions. The constellation is constantly on and does not require an acquisition planning. PlanetScope is commercial data and has to be ordered by the user",
 'extent': {'spatial': {'bbox': [[-180, -90, 180, 90]]},
  'temporal': {'interval': [['2016-01-01T00:00:00Z', None]]}},
 'id': 'PLANETSCOPE',
 'keywords': ['sentinel hub', 'PlanetScope', 'vhr', 'commercial data'],
 'license': 'various',
 'links': [],
 'providers': [{'description': '',
   'name': 'Sentinel Hub',
   'roles': ['processor'],
   'url': 'https://docs.sentinel-hub.com/api/latest/data/planet-scope/'},
  {'description': '',
   'name': 'Planet',
   

**The `PLANETSCOPE`collection is commercial data that has been acquired specifically for this Use Case through the ESA Networ of Resources (NOR), it is not openly accessible to all users.**

It is required to use a specific FeatureFlag denominated **BYOC** (Bring your own collection). It is read from a textfile that cannot be shared publicly.  
This means that **the target variable generation for the FCC use case is not reproducible for all users without the corrisponding BYOC code**.

In [5]:
byoc_id_file = "extdata/byoc.txt"
byoc_id  = open(byoc_id_file,"r").read().splitlines()[0]

## External Data

In order to process all the test sites a shapefile is read containing the boundaries of the smaller test areas. We applied a 0.0001° buffer in order to obtain all Pixels in the test sites

In [6]:
shp_path = "extdata/SuitableSitesVHR_selected_country.shp"
aoi_geometries = gpd.read_file(shp_path)
aoi_buffered_geometries = aoi_geometries.buffer(0.0001)

/home/mclaus@eurac.edu/anaconda3/envs/openeo_platform/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
aois_bounds = aoi_buffered_geometries.bounds.to_numpy()
print("Bounds of the first considered AOI in lat/lon (WGS84): ",aois_bounds[0])

Bounds of the first considered AOI in lat/lon (WGS84):  [ 7.79003931 50.30665141  7.79601901 50.310554  ]


## Connect to VHR data

In this exemplary Notebook the target variable is calculated for the first test site in Germany for the year 2018

In [8]:
aoi1_bounds = aois_bounds[0]
year = 2018

It might be difficult to use  to properly distinguish between forests and other land cover classes, especially those that also represent vegetation such as crops or grasslands. *Yang et al. (2019)* showed a nice representation of standardized forest NDVI signatures that nicely show the seasonal trend within a year based on different forest types. There are several time intervals of interest when generating a forest mask. Due to the high frequency of revisits by the `PLANETSCOPE` satellites each time interval is represented multiple times with several images. The time intervals of interest are:
- **summer_time**: Time of the maximum productivity of vegetation from mid April to mid September
- **winter_time**: Time of the minimum productivity of vegetation from mid November to mid February
- **annual_time**: The whole year range
- **total_time**: Takes into account the **annual_year** as well as some shoulder values in Winter (until mid-february the following year)

In [9]:
summer_time =  [np.datetime64(str(year)+"-04-15"),np.datetime64(str(year)+"-09-15")]
winter_time =  [np.datetime64(str(year)+"-11-15"),np.datetime64(str(year+1)+"-02-15")]
annual_time =  [np.datetime64(str(year)+"-01-01"),np.datetime64(str(year)+"-12-31")]
total_time  =  [np.datetime64(str(year)+"-01-01"),np.datetime64(str(year+1)+"-02-15")]

The `PLANETSCOPE` data is loaded primarily for **total_time** before the NDVI is calculated. Afterwards a clear-sky mask is applied to exclude all Pixel that are altered by clouds, haze and other atmospheric effects.

In [10]:
plnt = connection.load_collection(
    collection_id  = "PLANETSCOPE",
    spatial_extent = {"west": aoi1_bounds[0], "south": aoi1_bounds[1], "east": aoi1_bounds[2], "north": aoi1_bounds[3]},
    temporal_extent= [str(total_time[0]), str(total_time[1])]
    )
plnt._pg.arguments['featureflags'] = {'byoc_collection_id': byoc_id}

plnt_ndvi = plnt.ndvi(nir="B4",red="B3")
mask      = plnt.band("UDM2_Clear").apply(lambda x: x.neq(1))
plnt_ndvi_msk = plnt_ndvi.mask(mask=mask)

## Create Seasonal masks

### Summer

In this step the **summer_time** is applied to the data using the `filter_temporal` process. For analysing the whole summer period based on `median` and `sd` metrics, the `reduce_dimension` process is used on the *t* dimension.

We calculate two indicators important for the mask generation:
- **s_msk_med_hig**: Median summer NDVI above 0.6
- **s_msk_sd_low**: Summer NDVI standard deviation below 0.1

In [15]:
summer_time =  [np.datetime64(str(year)+"-04-15"),np.datetime64(str(year)+"-09-15")]
plnt_summer =  plnt_ndvi_msk.filter_temporal(extent= [str(summer_time[0]), str(summer_time[1])])

s_msk_med     = plnt_summer.reduce_dimension(dimension='t',reducer=median)
s_msk_med_hig = s_msk_med > 0.6

s_msk_sd      = plnt_summer.reduce_dimension(dimension='t',reducer=sd)
s_msk_sd_low  = s_msk_sd < 0.1

### Winter

In this step the **winter_time** is applied to the data using the `filter_temporal` process. For analysing the whole summer period based on `median` metric, the `reduce_dimension` process is used on the *t* dimension.

Here we calculate two indicators important for the mask generation:
- **w_msk_med_hig**: Median summer NDVI above 0.6
- **w_msk_med_low**: Median summer NDVI between 0 and 0.4

In [16]:
winter_time =  [np.datetime64(str(year)+"-11-15"),np.datetime64(str(year+1)+"-02-15")]
plnt_winter =  plnt_ndvi_msk.filter_temporal(extent= [str(winter_time[0]), str(winter_time[1])])

w_msk_med     = plnt_winter.reduce_dimension(dimension='t',reducer=median)
w_msk_med_hig = w_msk_med > 0.6

w_low_upper   = w_msk_med < 0.4
w_low_lower   = w_msk_med > 0
w_msk_med_low = w_low_upper * w_low_lower

### Year

In this step the **year_time** is applied to the data using the `filter_temporal` process. For analysing the whole summer period based on `median` metric, the `reduce_dimension` process is used on the *t* dimension.

Here we calculate two indicators important for the mask generation:
- **y_msk_med_hig**: Median yearly NDVI above 0.6

In [17]:
annual_time =  [np.datetime64(str(year)+"-01-01"),np.datetime64(str(year)+"-12-31")]
plnt_year   =  plnt_ndvi_msk.filter_temporal(extent= [str(annual_time[0]), str(annual_time[1])])

y_msk_med     = plnt_year.reduce_dimension(dimension='t',reducer=median)
y_msk_med_hig = y_msk_med > 0.6

## Create thematic masks

Based on the **seasonal_masks** also thematic ones are created:

- **f_evergreen_mask**: A mask with high summer and high winter values typically for evergreen (conifer) forests
- **f_deciduous_mask**: A mask with high summer and very low winter values typically hinting at deciduous forests
- **f_mixed_mask**: Mixed forests typically have a high yearly and high summer value but are the most difficult to spot. This mask sustains the other to exlude other vegetation types

In [18]:
f_evergreen_mask  = s_msk_med_hig * w_msk_med_hig
f_deciduous_mask  = s_msk_med_hig * w_msk_med_low
f_mixed_mask      = y_msk_med_hig * s_msk_med_hig

Finally the masks are being combined to a final forest canopy cover mask with th following criteria:
- **cmask_forest**: Estimates whether it is either an evergreen, deciduous forest or mixed (or multiple of those)
- **cmask_forest_error**: Excludes Pixel with a high standard deviation in summer. These are often hinting at management processes typical for crops and grassland vegetation

In [19]:
cmask_forest = f_evergreen_mask + f_deciduous_mask + f_mixed_mask
cmask_forest = cmask_forest > 0
cmask_forest_sd = cmask_forest * s_msk_sd_low

## Resample

The target variable will be resampled to 60m. The output pixels will be the result of the average over 20 x 20 (400) Planet pixels, since the Planet pixel size is 3 meters.

The 60m x 60m unit area corresponds to 6x6 (36) Sentinel-2 pixels with 10 meters resolution and 3x3 (9) Sentinel-1 pixels with 20m resolution.

This allows for a more stable estimation of the target variable during the random Forest regression

In [20]:
test_res = cmask_forest_sd.resample_spatial(resolution=60,method="average")
res_save = test_res.save_result(format="NetCDF")

In [17]:
job = res_save.send_job(title = "VH0_result_Resample60_average")
job.start_job()

In [21]:
job

<RESTJob job_id=''>

In [23]:
results = job.get_results()
results.download_files("./UC8_data/")

[PosixPath('UC8_data/openEO.nc')]

## Target vector-cube definition

In order to use the obtained canopy cover percentage in the following random forest regression step, we need firstly to create a vector-cube containing the geometries (60mx60m square polygons) with associated value of our target variable.

The following example performs this conversion for a single AOI. The full use case considers 150 AOIs.

Read the obtained netCDF from the previous openEO job

In [69]:
aoi1_target_variable = xr.open_dataset("./UC8_data/openEO.nc")
print(aoi1_target_variable)

<xarray.Dataset>
Dimensions:  (x: 12, y: 12)
Coordinates:
  * x        (x) float64 4.139e+05 4.139e+05 4.14e+05 ... 4.145e+05 4.146e+05
  * y        (y) float64 5.574e+06 5.574e+06 5.574e+06 ... 5.573e+06 5.573e+06
Data variables:
    crs      |S1 ...
    var      (y, x) float32 ...
Attributes:
    Conventions:  CF-1.8
    institution:  openEO platform - Geotrellis backend: 0.6.1a1
    description:  
    title:        


Filter out areas with no data

In [70]:
aoi1_target_variable_filtered = aoi1_target_variable['var'].where(aoi1_target_variable['var']!=0,drop=True).fillna(0)
print(aoi1_target_variable_filtered)

<xarray.DataArray 'var' (y: 7, x: 7)>
array([[0.47165534, 0.61904764, 0.63038546, 0.5873016 , 0.6666667 ,
        0.6666667 , 0.20408164],
       [0.6991342 , 1.        , 1.        , 1.        , 1.        ,
        1.        , 0.28138527],
       [0.00453515, 0.5735931 , 0.95238096, 1.        , 1.        ,
        1.        , 0.292517  ],
       [0.        , 0.01082251, 0.37641722, 1.        , 0.7900433 ,
        1.        , 0.2993197 ],
       [0.        , 0.        , 0.06709956, 0.82900435, 0.45041323,
        0.77922076, 0.36796537],
       [0.10204082, 0.01082251, 0.02494331, 0.6893424 , 0.478355  ,
        0.03174603, 0.10204082],
       [0.39002267, 0.01082251, 0.00226757, 0.25396827, 0.4004329 ,
        0.47165534, 0.08390023]], dtype=float32)
Coordinates:
  * x        (x) float64 4.139e+05 4.139e+05 4.14e+05 ... 4.142e+05 4.143e+05
  * y        (y) float64 5.574e+06 5.574e+06 5.574e+06 ... 5.574e+06 5.573e+06
Attributes:
    long_name:     var
    units:         
    grid_mappi

In this case, we end up with 7x7 pixels with a resolution of 60m each.

Each pixel has an associated value for the canopy coverage, which represents the average obtained over the PLANET data.

### UTM to EQUI7 reprojection

Since the Sentinel-2 and Sentinel-1 data we will use later as predictors (also known as features) are projected in EQUI7, we have to reproject the obtained data from UTM to obtain a vector-cube with geometries expressed in EQUI7.

This will allow us to be as precise as possible selecting the training pixels.

In [71]:
input_data_crs = aoi1_target_variable.crs.attrs["crs_wkt"]
equi7_wkt = "PROJCRS[\"Azimuthal_Equidistant\",BASEGEOGCRS[\"WGS 84\",DATUM[\"World Geodetic System 1984\",ELLIPSOID[\"WGS 84\",6378137,298.257223563,LENGTHUNIT[\"metre\",1]]],PRIMEM[\"Greenwich\",0,ANGLEUNIT[\"degree\",0.0174532925199433]],ID[\"EPSG\",4326]],CONVERSION[\"Modified Azimuthal Equidistant\",METHOD[\"Modified Azimuthal Equidistant\",ID[\"EPSG\",9832]],PARAMETER[\"Latitude of natural origin\",53,ANGLEUNIT[\"degree\",0.0174532925199433],ID[\"EPSG\",8801]],PARAMETER[\"Longitude of natural origin\",24,ANGLEUNIT[\"degree\",0.0174532925199433],ID[\"EPSG\",8802]],PARAMETER[\"False easting\",5837287.81977,LENGTHUNIT[\"metre\",1],ID[\"EPSG\",8806]],PARAMETER[\"False northing\",2121415.69617,LENGTHUNIT[\"metre\",1],ID[\"EPSG\",8807]]],CS[Cartesian,2],AXIS[\"easting\",east,ORDER[1],LENGTHUNIT[\"metre\",1,ID[\"EPSG\",9001]]],AXIS[\"northing\",north,ORDER[2],LENGTHUNIT[\"metre\",1,ID[\"EPSG\",9001]]]]"

In [72]:
target_canopy_cover_60m_equi7 = aoi1_target_variable_filtered.rio.write_crs(input_data_crs).rio.reproject(equi7_wkt)
target_canopy_cover_60m_equi7 = target_canopy_cover_60m_equi7.where(target_canopy_cover_60m_equi7<target_canopy_cover_60m_equi7.attrs["_FillValue"])
target_canopy_cover_60m_equi7.attrs["_FillValue"] = np.nan
print(target_canopy_cover_60m_equi7)

<xarray.DataArray 'var' (y: 8, x: 8)>
array([[       nan, 0.47165534, 0.61904764, 0.63038546,        nan,
               nan,        nan,        nan],
       [       nan, 0.6991342 , 1.        , 1.        , 0.5873016 ,
        0.6666667 , 0.6666667 , 0.20408164],
       [       nan, 0.00453515, 0.5735931 , 0.95238096, 1.        ,
        1.        , 1.        , 0.28138527],
       [       nan, 0.        , 0.01082251, 0.95238096, 1.        ,
        1.        , 1.        , 0.292517  ],
       [0.        , 0.        , 0.        , 0.37641722, 1.        ,
        0.7900433 , 1.        , 0.2993197 ],
       [0.10204082, 0.01082251, 0.02494331, 0.82900435, 0.45041323,
        0.77922076, 0.36796537,        nan],
       [0.39002267, 0.01082251, 0.00226757, 0.6893424 , 0.478355  ,
        0.03174603, 0.10204082,        nan],
       [       nan,        nan,        nan, 0.25396827, 0.4004329 ,
        0.47165534, 0.08390023,        nan]], dtype=float32)
Coordinates:
  * x        (x) float64 4.69

We have obtained our target canopy cover with EQUI7 projection. However, this is still not enough.

The reprojected data is on a grid which does not match the one used for the Sentinel-2 and Sentinel-1 data, which have the pixels on a grid with multiples of 10s. (This could be easily verified by inspecting some data retrieved from the collection `boa_sentinel_2`)

We will use the information of the spatial extent of reprojected data to generate a regularly spaced EQUI7 grid:

In [73]:
x_samples = len(target_canopy_cover_60m_equi7.x)
y_samples = len(target_canopy_cover_60m_equi7.y)
print("Number of samples along x: ",x_samples)
print("Number of samples along y: ",y_samples)

x_res = 60
y_res = 60
print("Resolution in x and y: {} m {} m".format(x_res,y_res))

equi7_x_min = target_canopy_cover_60m_equi7.x.min().values
equi7_x_max = target_canopy_cover_60m_equi7.x.max().values

equi7_y_min = target_canopy_cover_60m_equi7.y.min().values
equi7_y_max = target_canopy_cover_60m_equi7.y.max().values

equi7_x_min_regular = int(np.floor(equi7_x_min - equi7_x_min%5))
equi7_y_min_regular = int(np.floor(equi7_y_min - equi7_y_min%5))
equi7_y_max_regular = int(np.floor(equi7_y_max - equi7_y_max%5))

if equi7_x_min_regular%10 != 5:
    equi7_x_min_regular = equi7_x_min_regular+5
if equi7_y_min_regular%10 != 5:
    equi7_y_min_regular = equi7_y_min_regular+5

equi7_x_regular_coords = np.arange(equi7_x_min_regular,equi7_x_min_regular + x_samples*x_res,x_res)
equi7_y_regular_coords = np.arange(equi7_y_min_regular + y_samples*y_res,equi7_y_min_regular,-y_res)

print("Input EQUI7 x pixels center",target_canopy_cover_60m_equi7.x[0].values,target_canopy_cover_60m_equi7.x[-1].values)
print("Input EQUI7 y pixels center",target_canopy_cover_60m_equi7.y[0].values,target_canopy_cover_60m_equi7.y[-1].values)

print("Output EQUI7 x pixels center",equi7_x_regular_coords[0],equi7_x_regular_coords[-1])
print("Output EQUI7 y pixels center",equi7_y_regular_coords[0],equi7_y_regular_coords[-1])

Number of samples along x:  8
Number of samples along y:  8
Resolution in x and y: 60 m 60 m
Input EQUI7 x pixels center 4691619.964310119 4692070.14080146
Input EQUI7 y pixels center 1950913.0868664586 1950462.910375118
Output EQUI7 x pixels center 4691615 4692035
Output EQUI7 y pixels center 1950945 1950525


In [74]:
target_canopy_cover_60m_equi7_regular = xr.DataArray(
    dims=["x", "y"],
    coords=dict(
        y=(["y"], equi7_y_regular_coords),
        x=(["x"], equi7_x_regular_coords)
    )
).rio.write_crs(equi7_wkt)
target_canopy_cover_60m_equi7_regular

<xarray.DataArray (x: 8, y: 8)>
array([[nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan]])
Coordinates:
  * y            (y) int64 1950945 1950885 1950825 ... 1950645 1950585 1950525
  * x            (x) int64 4691615 4691675 4691735 ... 4691915 4691975 4692035
    spatial_ref  int64 0

`target_canopy_cover_60m_equi7_regular` contains our target datacube with a regularly spaced EQUI7 grid with 60 meters resolution: we can reproject the input UTM data to match it's projection and resolution.

In [78]:
target_canopy_cover_60m_equi7_regular = aoi1_target_variable_filtered.rio.write_crs(input_data_crs).rio.reproject_match(target_canopy_cover_60m_equi7_regular,1)
target_canopy_cover_60m_equi7_regular = target_canopy_cover_60m_equi7_regular.where(target_canopy_cover_60m_equi7_regular<target_canopy_cover_60m_equi7_regular.attrs["_FillValue"])
target_canopy_cover_60m_equi7_regular.attrs["_FillValue"] = np.nan

### Raster-cube to vector-cube conversion

The last step consists in a coversion from a raster, where each pixel covers an area of 60mx60m and has a float values associated (the target canopy cover), into multiple polygons of size 60mx60m with an associated float value.

In [76]:
output_vector_cube = gpd.GeoDataFrame(columns = ['geometry','target_canopy_cover'])

for x_s in target_canopy_cover_60m_equi7_regular.x.values:
    for y_s in target_canopy_cover_60m_equi7_regular.y.values:
        val = target_canopy_cover_60m_equi7_regular.loc[dict(x=x_s,y=y_s)].values
        if not np.isnan(val):
            x_min = x_s - 30
            x_max = x_s + 30
            y_min = y_s - 30
            y_max = y_s + 30
            polygon = Polygon([[x_min, y_min], [x_max, y_min], [x_max, y_max], [x_min, y_max]])
            output_vector_cube = output_vector_cube.append({'geometry':polygon,'target_canopy_cover':float(val)},ignore_index=True)
output_vector_cube

,geometry,target_canopy_cover
0,"POLYGON ((4691585.000 1950555.000, 4691645.000...",0.155453
1,"POLYGON ((4691585.000 1950495.000, 4691645.000...",0.390023
2,"POLYGON ((4691645.000 1950855.000, 4691705.000...",0.569452
3,"POLYGON ((4691645.000 1950795.000, 4691705.000...",0.461239
4,"POLYGON ((4691645.000 1950735.000, 4691705.000...",0.006097
5,"POLYGON ((4691645.000 1950675.000, 4691705.000...",0.001771
6,"POLYGON ((4691645.000 1950615.000, 4691705.000...",0.005357
7,"POLYGON ((4691645.000 1950555.000, 4691705.000...",0.049059
8,"POLYGON ((4691645.000 1950495.000, 4691705.000...",0.092787
9,"POLYGON ((4691705.000 1950855.000, 4691765.000...",0.625446


We have obtained a vector-cube which can be now written as a geoJSON (following the old specification allowing to specify a CRS and using EQUI7 projected coordinates).

We need to convert the geopandas GeoDataframe into a Python dict and manually add the "crs" field specifying the EQUI7 projection.

A geopandas bug prevent us to write a geoJSON with a WKT string as a CRS and therefore we have to add it manually:

In [77]:
output_vector_cube_dict = json.loads(output_vector_cube.to_json())
output_vector_cube_dict["crs"] = { "type": "name", "properties": { "name": equi7_wkt } }
with open("target_canopy_cover_equi7_60m.geojson","w") as out_geojson:
    json.dump(output_vector_cube_dict, out_geojson,indent=2)

The obtained geoJSON will be used in the next UC8 notebook in the `aggregate_spatial` and `fit_regr_random_forest` processes.